In [3]:
#Basic Import
import pandas as pd
import numpy as np
import requests
import json
import keyboard
import os
import threading
import signal
import sys
import time
from concurrent.futures import ThreadPoolExecutor
from bs4 import BeautifulSoup
import re
#Import for Selenium
import logging

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException
search_url = 'https://danbolig.dk/soeg/?o=newest-desc'

#To Break while loops
is_active = True
list_of_links = []

# Create Selenium functions

**save_webriver_content:** save the 

**break_on_escape:** when escape is pressed, the scroller stops.

**selenium_scroller:** open Chrome winow, wait 15 seconds, scroll to bottom. Return links



In [2]:
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    filename='script_log.txt',
                    filemode='w')
def save_links(content):
    with open('list_of_links.txt', 'w') as file:
        file.write('\n'.join(content) + ',' + '\n')
        
def pause_program():
    global is_active
    is_active = False

def on_escape_press(event):
    if event.name == 'esc':
        pause_program()

keyboard.on_press(on_escape_press)

def get_links(driver):
    links = driver.find_elements(By.XPATH, "//a[starts-with(@href, '/bolig/')]")
    list_of_links = [link.get_attribute('href') for link in links]
    return list_of_links

def selenium_scroller(search_url):
    global is_active
    is_active = True
    driver = webdriver.Chrome()
    url = search_url
    driver.get(url)
    scroll_height = 0
    last_height = driver.execute_script("return document.body.scrollHeight")
    time.sleep(15)
    property_listings_div = driver.find_element(By.CSS_SELECTOR, 'div.t-search__list')
    #list_of_links = []
    while is_active:
        try:
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", property_listings_div)
            time.sleep(1)
            new_scroll_height = driver.execute_script("return arguments[0].scrollHeight;", property_listings_div)
            if new_scroll_height == scroll_height:
                break
            scroll_height = new_scroll_height
        except KeyboardInterrupt:
            pause_program()
            break  
        except WebDriverException as e:
            logging.error(f"Selenium error: {e}")
            driver.save_screenshot("screenshot.png")
        except Exception as e:
            logging.exception("An unexpected error occurred")
            driver.save_screenshot("error_screenshot.png")
        except WebDriverException as e:
            logging.error(f"Selenium error: {e}")
            driver.save_screenshot("screenshot.png")
            driver.quit()  # Close the driver
        except Exception as e:
            logging.exception("An unexpected error occurred")
            driver.save_screenshot("error_screenshot.png")
            driver.quit()  # Close the driver
    list_of_links = get_links(driver)
    # Collect links and content
    
    
#    driver.quit()  # Close the driver
    
    return list_of_links
 


## Get, save and print list of links
Press escape to break scrolling.

In [ ]:
content = selenium_scroller(search_url)
save_links(content)
for i in content:
    print(i)

https://danbolig.dk/bolig/vejle/7100/villa/0950001202-095/
https://danbolig.dk/bolig/kolding/6000/lejlighed/0170000991-017/
https://danbolig.dk/bolig/aarhus/8200/lejlighed/2730000448-273/
https://danbolig.dk/bolig/kerteminde/5290/villa/2590001307-259/
https://danbolig.dk/bolig/vordingborg/4760/villa/2040000675-204/
https://danbolig.dk/bolig/mariagerfjord/9500/villa/4750000371-475/
https://danbolig.dk/bolig/halsnaes/3390/villa/2930000213-293/
https://danbolig.dk/bolig/vesthimmerlands/9600/villa/2580000393-258/
https://danbolig.dk/bolig/odense/5260/villa/2590001283-259/
https://danbolig.dk/bolig/ishoej/2635/villa/2760000207-276/
https://danbolig.dk/bolig/varde/6862/fritidsbolig/1140000327-114/
https://danbolig.dk/bolig/slagelse/4200/fritidsbolig/2840000399-284/
https://danbolig.dk/bolig/aarhus/8210/villa/2820000519-282/
https://danbolig.dk/bolig/albertslund/2620/lejlighed/2750000492-275/
https://danbolig.dk/bolig/rudersdal/2970/villa/1980000832-198/
https://danbolig.dk/bolig/roskilde/413

## Load the data from the links
Get zip code by regex, to separate roadname, zipcode and city in data


In [13]:
re_zip = r'https://danbolig\.dk/bolig/([^/]+)/(\d+)/'

zip_list = []
table_large = []
table_left = []
table_address = []
top_info_table = []
house_description = []
house_zip = []
house_data = []
def save_dataframe(frame, file_name):
    csv_filename = '{}.csv'.format(file_name)

    # Export the DataFrame to CSV
    frame.to_csv(csv_filename, index=False)
def get_page_content(url_list):
    for url in url_list:
        match = re.search(re_zip, url)
        if match:
            city = match.group(1)
            zip_code = match.group(2)
            zip_list.append(zip_code)
        try:
            print(url)
            # Send a request to the URL and get the content of the webpage
            response = requests.get(url, headers={'name': 'August Jensen', 'email': 'blz957@alumni.ku.dk', 'institution': 'University of Copenhagen'})
            soup = BeautifulSoup(response.content, 'lxml')

            # Access the large main table and add it to the list of large_tables
            info_table_large = soup.find_all('table')[0].find('tbody')
            table_large.append(info_table_large)

            # Access the small table to the left and add it to the list of left_tables
            info_table_small_left = soup.find_all('table')[1].find('tbody')
            table_left.append(info_table_small_left)

            # Access the small info table at the top (address) and add it to the list of top_addresses
            upper_info_table = soup.find_all('div', class_='o-propertyHero__info o-propertyHero__info--desktop')[0]

            # Find the addresses
            addresses = upper_info_table.find('h1')
            table_address.append(addresses)

            # Find house price and other stats
            info_table = upper_info_table.find_all('ul', class_='o-propertyHero__facts')[0]
            top_info_table.append(info_table)

            #Get house description
            description_tags = soup.find_all('div', class_='o-content o-propertyDescription__description a-paragraph')
            print("Number of description tags found:", len(description_tags)) 
            
            for description in description_tags:
                house_description.append(description.get_text(strip=True))
            if len(description_tags) == 0:
                house_description.append('No description available')
            time.sleep(1)
        except IndexError:
            print("IndexError: Skipping this link.")
            continue



def pass_content_to_dataframe():

    for house in range(len(table_large)):
        main_table_dict = {}

        main_table = table_large[house].find_all('td')
        for i in range(0, len(main_table), 2):
            name = main_table[i].text
            value = main_table[i + 1].text
            value = re.sub(r'\s*(m²|kr\.)', '', value)
            main_table_dict[name] = value

        top_table = top_info_table[house].find_all('span')
        for i in range(0, len(top_table), 2):
            name = top_table[i].text
            value = top_table[i + 1].text
            value = re.sub(r'\s*(m²|kr\.)', '', value)
            if name != 'Energimærke':
                main_table_dict[name] = value.strip()

        name = "address"
        value = ' '.join(table_address[house].text.split())
        main_table_dict[name] = value

        # Add description and zip_code fields from the provided lists
        main_table_dict['description'] = house_description[house]
        main_table_dict['zip code'] = zip_list[house]

        house_data.append(main_table_dict)

    df = pd.DataFrame(house_data)
    # Assuming there's a function save_dataframe that saves the DataFrame to a file
    save_dataframe(df, 'content')
    return df

def get_correct_address(row):
    address_parts = row['address'].split()
    try:
        # Find the index of the zip code in the address parts
        zip_index = address_parts.index(row['zip code'])
        
        # Join the parts to the left of the zip code as the road
        road = ' '.join(address_parts[:zip_index])
        
        # Join the parts to the right of the zip code as the city
        city = ' '.join(address_parts[zip_index + 1:])
    except ValueError:
        city = address_parts[-1]
        zip_code = address_parts[-2]
        
        # If the city is followed by a single letter, adjust parts accordingly
        if len(city) == 1 and len(address_parts) >= 3:
            city = address_parts[-2] + ' ' + city
            zip_code = address_parts[-3]
            road = ' '.join(address_parts[:-3])
        else:
            road = ' '.join(address_parts[:-2])
    return pd.Series({'City': city, 'zip code': row['zip code'], 'Road': road})

## Get information on houses
Then edit adress format, and save to csv file


In [7]:
get_page_content(content)
df = pass_content_to_dataframe()

https://danbolig.dk/bolig/vejle/7100/villa/0950001202-095/
Number of description tags found: 1
https://danbolig.dk/bolig/kolding/6000/lejlighed/0170000991-017/
Number of description tags found: 1
https://danbolig.dk/bolig/aarhus/8200/lejlighed/2730000448-273/
Number of description tags found: 1
https://danbolig.dk/bolig/kerteminde/5290/villa/2590001307-259/
Number of description tags found: 1
https://danbolig.dk/bolig/vordingborg/4760/villa/2040000675-204/
Number of description tags found: 1
https://danbolig.dk/bolig/mariagerfjord/9500/villa/4750000371-475/
Number of description tags found: 1
https://danbolig.dk/bolig/halsnaes/3390/villa/2930000213-293/
Number of description tags found: 1
https://danbolig.dk/bolig/vesthimmerlands/9600/villa/2580000393-258/
Number of description tags found: 1
https://danbolig.dk/bolig/odense/5260/villa/2590001283-259/
Number of description tags found: 1
https://danbolig.dk/bolig/ishoej/2635/villa/2760000207-276/
Number of description tags found: 1
https

In [17]:
df[['City', 'zip code', 'Road']] = df.apply(get_correct_address, axis=1)
save_dataframe(df, 'content-2')
df

# Geotag and Distance Parameter


In [24]:
from geopy.geocoders import Nominatim
import time
df = pd.read_csv('content.csv')

# Initialize Nominatim API
geolocator = Nominatim(user_agent="MyApp")
from geopy.geocoders import GoogleV3
df['Road1'] = df['Road'].str.split(',', 1).str[0]
df['City'] = df['City'].astype(str)
df['full_address'] = df['Road1'] + ' ' + df['City']
df['full_address'] = df['full_address'].astype(str)
df['latitude'] = 0
df['longitude'] = 0

count = 0 
for index, row in df.iterrows():
    address = row['full_address']
    try:
        location = geolocator.geocode(address)
        if location is not None:
            df.at[index, 'latitude'] = location.latitude
            df.at[index, 'longitude'] = location.longitude
            print( index, location.longitude, location.latitude)
        else:
            df.at[index, 'latitude'] = 'Not available'
            df.at[index, 'longitude'] = 'Not Available'
            print(address)
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
    #time.sleep(1)

save_dataframe(df, 'content-3')

C:\Users\Augus\AppData\Local\Temp\ipykernel_21292\62224008.py:8: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df['Road1'] = df['Road'].str.split(',', 1).str[0]


0 9.546171 55.699203
1 9.48867 55.49514
2 10.216531 56.16927
3 10.521027 55.387506
4 11.920209 55.00674
5 9.787053 56.648091
6 11.864029 55.95423
7 9.431766 56.84419
8 10.425587 55.34424
9 12.346797 55.625952
10 8.647337 55.692607
11 11.208483 55.439098
12 10.169554 56.165147
13 12.341502 55.670677
14 12.477897 55.873301
15 12.022436 55.552349
16 12.095486 55.570284
17 12.564396 55.671331
18 11.736612 55.226137
19 10.311852 56.994833
20 12.127343 55.655312
21 10.150169 56.150395
22 11.74414 55.233074
23 12.414721 55.969334
24 9.697474 54.951926
25 10.092694 56.232042
26 11.059458 55.698094
27 12.084871 55.639832
28 8.86365 56.037659
29 11.956876 55.313836
30 12.11736 55.242986
31 8.893555 56.131521
32 9.419379 55.707411
33 12.502372 55.645051
34 9.557353 56.173182
35 14.709212 55.181213
36 9.92063 56.045366
37 14.759169 55.139867
38 12.048555 54.980426
39 8.687052 56.469327
40 11.213762 55.437427
41 12.371322 55.791578
42 9.76548 55.542425
43 11.908971 54.890219
44 12.536467 55.755195


# Get nearest city

In [35]:
from geopy.distance import geodesic
import pandas as pd
import numpy as np

# Sample cities and their coordinates
cities = {
    'Copenhagen': (55.6761, 12.5683),
    'Aarhus': (56.1629, 10.2039),
    'Aalborg': (57.0480, 9.9197),
    'Odense': (55.3962, 10.3909)
}

def calculate_distance_to_city(row):
    geopoint = (row['latitude'], row['longitude'])
    min_distance = float('inf')  # Initialize with a large value
    closest_city = None

    for city, coordinates in cities.items():
        try:
            distance = geodesic(geopoint, coordinates).kilometers
        except ValueError:
            distance = np.nan
        if distance < min_distance:
            min_distance = distance
            closest_city = city
    
    return pd.Series({'distance to city': min_distance, 'closest city': closest_city})

# Load your DataFrame here
df = pd.read_csv('geo.csv')

# Apply the function to calculate distances and closest city, and merge the results
df[['distance to city', 'closest city']] = df.apply(calculate_distance_to_city, axis=1)

# Save the updated DataFrame
df.to_csv('updated_geo.csv', index=False)

In [36]:
df

,Unnamed: 0,Type,Udbudsform,Energimærke,Varmekilde,Byggeår,Rum,Bad,Toilet,Plan,...,Kælderum,garage,City,Road,Road1,full_address,latitude,longitude,distance to city,closest city
0,0,Villa,Salg,C,Naturgas,1976.0,4.0,2.0,2.0,2.0,...,NaN,NaN,Vejle,"Banevang 22, Mølholm",Banevang 22,Banevang 22 Vejle,55.699203,9.546171,63.093789,Odense
1,1,Lejlighed,Salg,E,Fjernvarme/blokvarme,1906.0,4.0,1.0,1.0,1.0,...,NaN,NaN,Kolding,"Castenskjoldsvej 3, 1.",Castenskjoldsvej 3,Castenskjoldsvej 3 Kolding,55.49514,9.48867,58.148090,Odense
2,2,Lejlighed,Salg,A2010,Fjernvarme,2011.0,4.0,1.0,1.0,1.0,...,NaN,NaN,Aarhus N,"Trøjborgvej 8T, 2. th., Trøjborg",Trøjborgvej 8T,Trøjborgvej 8T Aarhus N,56.16927,10.216531,1.057740,Aarhus
3,3,Villa,Salg,C,Centralvarme med én fyringsenhed,1998.0,5.0,2.0,2.0,1.0,...,NaN,NaN,Marslev,Alléparken 94,Alléparken 94,Alléparken 94 Marslev,55.387506,10.521027,8.302623,Odense
4,4,Villa,Salg,C,Fjernvarme,1959.0,7.0,2.0,2.0,2.0,...,NaN,NaN,Vordingborg,Drosselvej 2,Drosselvej 2,Drosselvej 2 Vordingborg,55.00674,11.920209,85.111579,Copenhagen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5065,5065,Lejlighed,Salg,D,Fjernvarme/blokvarme,1940.0,3.0,1.0,2.0,NaN,...,NaN,NaN,Gentofte,"Vangede Bygade 88, 1. th.",Vangede Bygade 88,Vangede Bygade 88 Gentofte,55.749989,12.517918,8.815002,Copenhagen
5066,5066,Villa,Salg,C,Oliefyr,1911.0,2.0,2.0,2.0,2.0,...,NaN,NaN,Videbæk,"Vorgod Østerbyvej 10, Vorgod Ø",Vorgod Østerbyvej 10,Vorgod Østerbyvej 10 Videbæk,56.065208,8.757102,90.656905,Aarhus
5067,5067,Villa,Salg,A2020,Varmepumpe,2017.0,6.0,2.0,2.0,1.0,...,NaN,NaN,Øst,"Frydenlund 82, Bredballe 7120",Frydenlund 82,Frydenlund 82 Øst,55.734018,9.618449,60.150220,Aarhus
5068,5068,Landejendom,Salg,G,Centralvarme med én fyringsenhed,1890.0,7.0,1.0,1.0,1.0,...,NaN,NaN,Stege,"Kirkeholmvej 15, Hjertebjerg",Kirkeholmvej 15,Kirkeholmvej 15 Stege,54.988697,12.404484,77.231215,Copenhagen


# DinGeo Data / tinglysning
